https://www.kaggle.com/laxmikantnishad/covid-19-2-para

In [1]:
is_local=False

yolov5_pred = True
effdet_pred = False
mmdet_pred = False

In [2]:
if not is_local:
    !conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
    !pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null
    
#     !pip install pylibjpeg 
    #pylibjpeg-libjpeg


######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | / - \ | / - \ | / - \ | / - \ | / done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done

#####################

In [3]:
if not is_local and mmdet_pred:
    ## MMDetection Offline Installation
    !pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
    !pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
    !pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
    !pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
    !pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
    !pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
    !pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

    !cp -r /kaggle/input/mmdetectionv2140/mmdetection-2.14.0 /kaggle/working/
    !mv /kaggle/working/mmdetection-2.14.0 /kaggle/working/mmdetection
    %cd /kaggle/working/mmdetection
    !pip install -e . --no-deps
    %cd /kaggle/working/

In [4]:
import sys; 

package_paths = [
    '../input/image-fmix/FMix-master'
]

if not is_local:
    package_paths.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
#     package_paths.append('../input/timm-efficientdet-pytorch')
#     package_paths.append('../input/effdet-latestvinbigdata-wbf-fused/efficientdet-pytorch')
    if effdet_pred:
        package_paths.append('../input/timmefficientdet024/effdet-0.2.4')
    package_paths.append('../input/omegaconf')
    package_paths.append('../input/weightedboxesfusion')
    if mmdet_pred:
        package_paths.append('/kaggle/working/mmdetection')
    #pytorch-image-models/pytorch-image-models-master

for pth in package_paths:
    sys.path.append(pth)

In [5]:
from glob import glob
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
# import torchvision
# from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import timm
import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
from scipy.ndimage.interpolation import zoom
# import pydicom

from fmix import sample_mask, make_low_freq_image, binarise_mask
from sklearn.model_selection import GroupKFold, StratifiedKFold

# import pydicom
# from pydicom.pixel_data_handlers.util import apply_voi_lut
from PIL import Image
from tqdm.auto import tqdm
import shutil
import ast

（1）dcm to .png

In [6]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    import pydicom
    from pydicom.pixel_data_handlers.util import apply_voi_lut
#     import pylibjpeg
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data
        

In [7]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [8]:
def prepare_test_images(img_size=512):
    split = 'test'
    save_dir = f'./{split}/'

    os.makedirs(save_dir, exist_ok=True)

    save_dir = f'./{split}/study/'
    os.makedirs(save_dir, exist_ok=True)

    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=img_size)  
            study = dirname.split('/')[-2] + '_study.png'
            #study = dirname.split('/')[-2] + '_study'
            im.save(os.path.join(save_dir, study))

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

#img = get_img('../input/siim-covid19-detection/test/00188a671292/3eb5a506ccf3/3dcdfc352a06.dcm')
#plt.imshow(img)
#plt.show()

In [10]:
class CustomDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True, mode='train',
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.mode = mode
        if self.mode == 'test':
            self.id_col_name = 'id'
        else:
            self.id_col_name = 'study_id'
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['class_id']
        
        tempstr = self.df.iloc[index][self.id_col_name][-3:]
        if tempstr == 'png':
            path = "{}/{}".format(self.data_root, self.df.iloc[index][self.id_col_name])
        else:
            path = "{}/{}".format(self.data_root, self.df.iloc[index][self.id_col_name])+'.png'
        img  = get_img(path)

        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

In [11]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            RandomResizedCrop(cfg['img_size'], cfg['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            CenterCrop(cfg['img_size'], cfg['img_size'], p=1.),
            Resize(cfg['img_size'], cfg['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms():
    return Compose([
            RandomResizedCrop(cfg['img_size'], cfg['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [12]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [13]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [14]:
def create_sub_df(sub_df, test, tst_preds):
#     if fast_sub:
#         sub_df = test.copy()
#     else:
#         sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    for i in range(len(test)):
        for j in range(len(test)):
            a = test.loc[i,'id'].split('.')[0]
            b = sub_df.loc[j,'id']
            if a==b:
                negative, typical, indeterminate, atypical = str(tst_preds[i][0]),str(tst_preds[i][1]),str(tst_preds[i][2]),str(tst_preds[i][3]),
                sub_df.loc[j,'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
    return sub_df

In [15]:
def cls_inference(cfg, is_local, is_ensemble=True, mode='multi'):
    if is_local:
        print('Inference on Validation dataset')
    else:
        print('Inference on Test dataset')

    seed_everything(cfg['seed'])
    # folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train_study.shape[0]), train_study.label.values)

    if is_local:
        if mode == 'multi':
            model_dir = f'../models/{cfg["exp_id"]}/weights/'
            train_img_path = '../input/siimcovid19-512-img-png-600-study-png/study/'
            test_img_path = './test/study/'
            train_csv_path = f'../input/siim_covid19_train/{cfg["train_csv_name"]}'
        elif mode == 'binary':
            model_dir = f'../models/{cfg["2cls_exp_id"]}/weights/'
            train_img_path = '../input/siimcovid19-512-img-png-600-study-png/image/'
            test_img_path = './test/image/'
            train_csv_path = f'../input/siim_covid19_train/train_image_level_2class.csv'
        train = pd.read_csv(train_csv_path)
    else:
        if mode == 'multi':
            model_dir = f'../input/siim-covid19-studyclass-models/{cfg["exp_id"]}/weights/'
            train_img_path = '../input/siimcovid19-512-img-png-600-study-png/study/'
            test_img_path = f'/kaggle/tmp/{split}/study/'
            train_csv_path = f'../input/siim-covid19-train/{cfg["train_csv_name"]}'
        elif mode == 'binary':
            model_dir = f'../input/siimcovid192classmodels/{cfg["2cls_exp_id"]}/weights/' 
            train_img_path = '../input/siimcovid19-512-img-png-600-study-png/image/'
            test_img_path = f'/kaggle/tmp/{split}/image/' # + test['id'] +'.png'
            train_csv_path = f'../input/siim-covid19-train/train_image_level_2class.csv'
#         test_img_path = './test/study/'
        
        if fast_sub:
            test = fast_df.copy()
        else:
            test = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
        
        
        if mode == 'multi':
            test = test[:study_len]
        elif mode == 'binary':
            test = test[study_len:]
        test['none'] = 0
        test.reset_index(inplace=True)
    
    accs = []
    test_preds_all = 0
    for idx, fold in enumerate(range(cfg['fold_num'])):
        # we'll train fold 0 first
#         if fold > 0:
#             break 

        print('Inference fold {} started'.format(fold))

        if is_local:
            valid_df = train[train['fold'] == fold]
            # valid_ = train_study.loc[val_idx,:].reset_index(drop=True)
            #print('valid_',valid_)
            valid_ds = CustomDataset(valid_df, train_img_path, transforms=get_inference_transforms(), output_label=False, mode='valid')
            val_loader = torch.utils.data.DataLoader(
                valid_ds, 
                batch_size=cfg['valid_bs'],
                num_workers=cfg['num_workers'],
                shuffle=False,
                pin_memory=False,
            )
            data_loader = val_loader
        else:
#             test = pd.DataFrame()
#             test['id'] = list(os.listdir(test_img_path))
            #print('test',test)
            test_ds = CustomDataset(test, test_img_path, transforms=get_inference_transforms(), output_label=False, mode='test')
            #print('test_ds',test)
            
            tst_loader = torch.utils.data.DataLoader(
                test_ds, 
                batch_size=cfg['valid_bs'],
                num_workers=cfg['num_workers'],
                shuffle=False,
                pin_memory=False,
            )
            data_loader = tst_loader

        
#         valid_df.class_id.nunique()

        if mode == 'multi':
            num_classes = cfg["num_classes"]
        elif mode == 'binary':
            num_classes = 2
        print(f'Num Class: {num_classes}')
        model = CassvaImgClassifier(cfg['model_arch'], num_classes).to(device)
        

        preds = []
#         ensemble_preds = []
        
        #for epoch in range(CFG['epochs']-3):
        # for i, epoch in enumerate(cfg['used_epochs']):    
        model_path = f'{model_dir}best_acc_{cfg["model_arch"]}_fold{fold}.pt'
        print(f'Loading {model_path}')
        model.load_state_dict(torch.load(model_path))
        
        with torch.no_grad():
            for _ in range(cfg['tta']):
#                 if is_ensemble:
#                     ensemble_preds += [cfg['weights'][0]/sum(cfg['weights'])/cfg['tta']*inference_one_epoch(model=model, data_loader=data_loader, device=device)]
#                 else:
                preds += [cfg['weights'][0]/sum(cfg['weights'])/cfg['tta']*inference_one_epoch(model=model, data_loader=data_loader, device=device)]
#         if is_ensemble:
#             ensemble_preds = np.mean(ensemble_preds, axis=0)
#         else:
        preds = np.mean(preds, axis=0)
    
        if is_ensemble:
            test_preds_all += preds
            
        if is_local:
            acc = (valid_df.class_id.values==np.argmax(preds, axis=1)).mean()
            print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_df.class_id.values, preds)))
            print('fold {} validation accuracy = {:.5f}'.format(fold, acc))
            accs.append(acc)
        
        del model
        gc.collect()
        torch.cuda.empty_cache()
    
    if not is_local:
        if is_ensemble:
            test_preds_all /= cfg['fold_num']
#             sumfile = create_sub_df(test, test_preds_all)
            return test, test_preds_all
        else:
#             sumfile = create_sub_df(test, preds)
            return test, preds
#         return sumfile
    else:
        mean_acc = (sum(accs)/cfg["fold_num"])*100.0
        print(f'Mean Accuracy {mean_acc}')

In [16]:
# !ls /kaggle/tmp/{split}/image/

# 4 Main

In [17]:
# if __name__ == '__main__':
cfg = {
    'exp_id': '20210725_01_50_04', #'20210726_17_17_07', #20210725_01_50_04', #'exp1',
    '2cls_exp_id': '20210729_01_58_48',
    'effdet_exp_id': '20210804_09_57_26',
    'train_csv_name':'train_classification_fold_v1.csv',
    'fold_num': 5,
    'num_classes': 4,
    'seed': 719,
    #'model_arch': 'tf_efficientnet_b7',
    'model_arch': 'tf_efficientnetv2_l_in21ft1k', #'tf_efficientnet_b4_ns', m_in21ft1k l_in21ft1k
    'img_size': 512,
    'epochs': 100,
    'train_bs': 14,
    'valid_bs': 32,
    'det_batch_size': 8,
    'T_0': 10,
    'lr': 1e-6,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 4,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 5, #5, #3,
    'used_epochs': [53,55,56,59],
    'weights': [1,1,1,1]
}

In [18]:
df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False
    
device = torch.device(cfg['device'])

## Make Sturdy Dir

In [19]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=600)  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=600)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=600)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))

## Make Image Dir

In [20]:
image_id = []
dim0 = []
dim1 = []
splits = []
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=cfg['img_size'])  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=cfg['img_size'])  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=cfg['img_size'])  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

In [21]:
if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
    
    
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

In [22]:
is_ensemble = True
if not is_local:
    test, test_preds = cls_inference(cfg, is_local, is_ensemble=is_ensemble, mode='multi')
else:
    cls_inference(cfg, is_local, is_ensemble=True, mode='multi')

Inference on Test dataset
Inference fold 0 started
Num Class: 4
Loading ../input/siim-covid19-studyclass-models/20210725_01_50_04/weights/best_acc_tf_efficientnetv2_l_in21ft1k_fold0.pt


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
if is_ensemble:
    test_preds /= cfg['fold_num']

sumfile = create_sub_df(df, test, test_preds)

In [24]:
df_study = sumfile[['id', 'PredictionString']]

In [25]:
del sumfile
gc.collect()

60

In [26]:
# if not yolov5_pred:
#     df_study.to_csv('/kaggle/working/submission.csv',index = False) 

## 2 Class

In [27]:
if not is_local:
    test_2cls, test_preds_2cls = cls_inference(cfg, is_local, is_ensemble=is_ensemble, mode='binary')
else:
    cls_inference(cfg, is_local, is_ensemble=True, mode='binary')

Inference on Test dataset
Inference fold 0 started
Num Class: 2
Loading ../input/siimcovid192classmodels/20210729_01_58_48/weights/best_acc_tf_efficientnetv2_l_in21ft1k_fold0.pt


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
if is_ensemble:
    test_preds_2cls /= cfg['fold_num']

In [29]:
label_cols = test_2cls.columns[3]
test_2cls[label_cols] = test_preds_2cls #sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
df_2class = test_2cls.reset_index(drop=True)

In [30]:
del test_2cls
gc.collect()

60

## Detection Configs

In [31]:
split = 'test'
test_dir =f'/kaggle/tmp/{split}/image'  #f'../{split}/study' #

In [32]:
# !ls /kaggle/tmp/test/image

In [33]:
if not is_local:
    meta = meta[meta['split'] == 'test']
    if fast_sub:
        test_df = fast_df.copy()
    else:
        test_df = pd.read_csv('/kaggle//input/siim-covid19-detection/sample_submission.csv')
    test_df = df[study_len:].reset_index(drop=True)
    test_df['image_path'] = test_dir + '/' + test_df['id']+'.png'

    meta['image_id'] = meta['image_id'] + '_image'
    meta.columns = ['id', 'dim0', 'dim1', 'split']
    test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

In [34]:
test_df.head()

,id,PredictionString,id_last_str,image_path,dim0,dim1,split
0,65761e66de9f_image,none 1 0 0 1 1,e,/kaggle/tmp/test/image/65761e66de9f_image.png,2330,2783,test
1,51759b5579bc_image,none 1 0 0 1 1,e,/kaggle/tmp/test/image/51759b5579bc_image.png,3093,2850,test


## YOLOv5 Inference

In [35]:
if yolov5_pred:
    def yolo2voc(image_height, image_width, bboxes):
        """
        yolo => [xmid, ymid, w, h] (normalized)
        voc  => [x1, y1, x2, y1]

        """ 
        bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

        bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
        bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

        bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
        bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

        return bboxes

In [36]:
if not is_local and yolov5_pred:
    if not os.path.exists('/kaggle/working/yolov5'):
        shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
    os.chdir('/kaggle/working/yolov5') # install dependencies

In [37]:
if yolov5_pred:
    dim = 512 #1024, 256, 'original'
    # weights_dir = '/kaggle/input/siim-cov19-yolov5-train/yolov5/runs/train/exp/weights/best.pt'
#     weights_dir = '/kaggle/input/siim-covid19-yolov5-model/exp8/weights/best.pt'
    YOLO_MODEL_PATHS = [
        '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-0/weights/best.pt',
        '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-1/weights/best.pt',
        '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-2/weights/best.pt',
        '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-3/weights/best.pt',
        '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-4/weights/best.pt',
        
        '/kaggle/input/siim-covid19-yolov5-model/batch16/yolov5x-img-512-fold-0/weights/best.pt',
        '/kaggle/input/siim-covid19-yolov5-model/batch16/yolov5x-img-512-fold-1/weights/best.pt',
        '/kaggle/input/siim-covid19-yolov5-model/batch16/yolov5x-img-512-fold-2/weights/best.pt',
        '/kaggle/input/siim-covid19-yolov5-model/batch16/yolov5x-img-512-fold-3/weights/best.pt',
        '/kaggle/input/siim-covid19-yolov5-model/batch16/yolov5x-img-512-fold-4/weights/best.pt',
    ]
    
    CONFIDENCE = [
        0.269, 0.268, 0.209, 0.179, 0.308
    ]

    #clear_output()
    #print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

#     !python detect.py --weights $weights_dir \
    !python detect.py --weights {YOLO_MODEL_PATHS[0]} {YOLO_MODEL_PATHS[1]} {YOLO_MODEL_PATHS[2]} {YOLO_MODEL_PATHS[3]} {YOLO_MODEL_PATHS[4]} {YOLO_MODEL_PATHS[5]} {YOLO_MODEL_PATHS[6]} {YOLO_MODEL_PATHS[7]} {YOLO_MODEL_PATHS[8]} {YOLO_MODEL_PATHS[9]} \
                        --img 512\
                        --conf 0.2 \
                        --iou 0.5 \
                        --source $test_dir \
                        --augment \
                        --save-txt --save-conf --exist-ok
    
    #                         --max-det 3 \
    # --conf 0.001\
    #                        --augment \

Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.2, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-0/weights/best.pt', '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-1/weights/best.pt', '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-2/weights/best.pt', '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-3/weights/best.pt', '/kaggle/input/siim-covid19-yolov5-model/batch16_ls02/yolov5x-img-512-fold-4/weights/best.pt', '/kaggle/input/siim-covid19-yolov5-model/batch16/yolov5x-img-512-fold-0/weights/best.pt', '/kaggle/input/siim-covid19-yolov5-model/batch16/yolov5x-img-512-fold-1/weights/best.pt', '/kaggle/input/siim-covid19-yolov5-model/batch16/yolov5x-img-512-fold-2/weight

In [38]:
if yolov5_pred:
    image_ids = []
    PredictionStrings = []

    for file_path in tqdm(glob('runs/detect/exp/labels/*.txt')):
        image_id = file_path.split('/')[-1].split('.')[0]
    #     print(image_id)
        if len(test_df[test_df['id']==image_id]) >= 1:
            w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
            f = open(file_path, 'r')
            data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
            data = data[:, [0, 5, 1, 2, 3, 4]]
            bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
            for idx in range(len(bboxes)):
                bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
            image_ids.append(image_id)
            PredictionStrings.append(' '.join(bboxes))


    pred_df = pd.DataFrame({'id':image_ids,
                            'PredictionString':PredictionStrings})

  0%|          | 0/2 [00:00<?, ?it/s]

## EfficientDet 

In [39]:
if effdet_pred:
    # --- ensemble boxes ---
    import ensemble_boxes
    from ensemble_boxes import *
#     import effdet.bench as bench
#     from effdet.anchors import MAX_DETECTION_POINTS

In [40]:
if effdet_pred:
    from effdet.config.model_config import efficientdet_model_param_dict
#     from effdet.config import model_config

    from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
    # DetBenchPredict DetBenchEval
    from effdet.efficientdet import HeadNet
    
    def get_test_transforms():
        return Compose([Resize(height=cfg['img_size'], width=cfg['img_size'], p=1.0),], p=1.0)
    
    
    class DetectDataset(Dataset):
        def __init__(self, test_df, test_dir, transform=None):
            super().__init__()
            self.test_df = test_df
            self.test_dir = test_dir
            self.transform = transform

        def __getitem__(self, idx):
            row = self.test_df.iloc[idx]
            img_id = row['id'] + '.png'
            img_path = self.test_dir  + '/' + img_id
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f'{img_path} not found')

            if self.transform:
                transformed = self.transform(image=img)
                img = transformed['image']

            # normalize img
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
            img /= 255.0

            # convert image into a torch.Tensor
            img = torch.as_tensor(img, dtype=torch.float32)
            #idx = torch.tensor([idx])

            # permute image to [C,H,W] from [H,W,C] and normalize
            img = img.permute(2, 0, 1)

            return img, img_id

        def __len__(self):
            return len(self.test_df)
    
    
    def create_model(num_classes=1, image_size=512, architecture="tf_efficientnetv2_l", checkpoint_path=''):
        efficientdet_model_param_dict[architecture] = dict(
            name=architecture,
            backbone_name=architecture,
            backbone_args=dict(drop_path_rate=0.2),
            num_classes=num_classes,
            url='', )

        config = get_efficientdet_config(architecture)
        config.update({'num_classes': num_classes})
        config.update({'image_size': (image_size, image_size)})

        net = EfficientDet(config, pretrained_backbone=False)
        
#         config.num_classes = num_classes
#         config.image_size = image_size
        net.class_net = HeadNet(config, num_outputs=config.num_classes)
        
#         print(config)
        # load net
        checkpoint = torch.load(checkpoint_path)
        if 'backbone.conv_stem.weight' not in checkpoint:
            print('model weight changing')
            model_state_dict = {}
            for k,v in checkpoint.items():
                if 'model.' == k[:6]:
                    k = k[6:]
                    model_state_dict[k] = v
        else:
            model_state_dict = checkpoint
        net.load_state_dict(model_state_dict)
        del checkpoint, model_state_dict
        gc.collect()
    
        net = DetBenchPredict(net)
#         model = DetBenchEval(net, config)
        net.eval()
        
        device = torch.device(cfg['device'])
        return net.to(device)

    
    def load_obj_detection_model(model_name, checkpoint_path):
        device = torch.device(cfg['device'])
        model = create_model(num_classes=1, image_size=cfg['img_size'], architecture=model_name)
        model.eval();
        checkpoint = torch.load(checkpoint_path)['model_state_dict']
        checkpoint['anchors.boxes'] = checkpoint.pop('anchor_labeler.anchors.boxes')
        model.load_state_dict(checkpoint)
        return model.to(device)


    # wbf = weighted boxes fusion for ensembling boxes from object detection models
    def run_wbf(predictions, img_idx, iou_thr=0.4, skip_box_thr=0.2, weights=None):
#     def run_wbf(predictions, img_idx, iou_thr=0.44, skip_box_thr=0.43, weights=None):

#         iou_thr=0.44, skip_box_thr=0.43
        # prediction[img_idx] is the prediction of each model for the same image (the image in img_idx)
        # weighted_boxes_fusion function received boxes with values in range [0-1]
        # normalize by dividing over cfg['img_size']-1 (includes 0, therefore -1)
        boxes = [(prediction[img_idx]['boxes']/(cfg['img_size']-1)).tolist() for prediction in predictions]
        scores = [prediction[img_idx]['scores'].tolist() for prediction in predictions]
        labels = [np.ones(prediction[img_idx]['scores'].shape[0]).astype(int).tolist() for prediction in predictions]
        boxes, scores, labels = ensemble_boxes.ensemble_boxes_wbf.weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        # scale back boxes by multiplying with cfg['img_size']-1
        boxes = boxes*(cfg['img_size']-1)
        return boxes, scores, labels


    # prediction for a single batch
    def batch_predictions(images, image_ids, score_threshold=0.22):
        with torch.no_grad():
            predictions = []
            images = images.float().cuda()

            # for each model get predictions for each image in batch
            for model in models:  
                model_predictions = []

                img_size = torch.tensor([images[0].shape[-2:]] * cfg['det_batch_size'], dtype=torch.float).to("cuda:0")

                img_info = {
                    'img_scale': torch.tensor([1]*images.shape[0]).float().cuda(),
                    'img_size': img_size
                }
                outputs = model(images, img_info=img_info)
                

                for i, (image, image_id) in enumerate(zip(images, image_ids)):
                    boxes = outputs[i].detach().cpu().numpy()[:,:4]    
                    scores = outputs[i].detach().cpu().numpy()[:,4]
                    
                    indexes = np.where(scores > score_threshold)[0]
                    boxes = boxes[indexes]
#                     boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
#                     boxes[:, 3] = boxes[:, 3] + boxes[:, 1]

#                     model_predictions.append({
#                     'boxes': boxes,
#                     'scores': scores
#                     })
                    model_predictions.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                    })
                # append model prediction to all predictions
                predictions.append(model_predictions)     

        return predictions

    def get_boxes(boxes, row, index):
        # get ratio to scale boxes
#         width = ast.literal_eval(row['dim1'].values[0])
#         height = ast.literal_eval(row['dim0'].values[0])
        width = row['dim1'].values[0]
        height = row['dim0'].values[0]
#         print(height, width)
#         width, height = raw.loc[raw.id==image_id,['dim1', 'dim0']].values[0]
#         height, width = original_shape[0], original_shape[1]
        h_ratio = height / cfg['img_size']
        w_ratio = width / cfg['img_size']

        # scale and convert from xywh to xyxy
        new_boxes = []
        for box in boxes:
            x,y,w,h = box
#             print(x,y,w,h)
            x1=x*w_ratio
            x2=(x+w)*w_ratio
            y1=y*h_ratio
            y2=(y+h)*h_ratio
            new_boxes.append(np.array([int(x1),int(y1),int(x2),int(y2)]))
            print('new', new_boxes)

        return np.array(new_boxes)    

    def ensemble_predictions(test_loader):
        # returns a df with img_id, study_id and predicted values (labels, scores and boxes) for each image
        img_ids = []
#         study_ids = []
        pred_boxes =[]
        pred_scores = []
        pred_labels = []

        for images, image_ids in tqdm(test_loader):
            # get batch predictions of all models
            # predictions[0] = batch predictions for model0, predictions[1] = batch predictions for model1 and etc...
            predictions = batch_predictions(images, image_ids)
            for i, (image, image_id) in enumerate(zip(images, image_ids)):
                # for each image get ensembled prediction by using wbf
                boxes, scores, labels = run_wbf(predictions, img_idx=i) 
                if '.png' in image_id:
                    image_id = image_id.replace('.png', '')
                row = test_df[test_df['id']==image_id]
                new_boxes = get_boxes(boxes, row, index=i)

                img_ids.append(image_ids[i])
#                 study_ids.append(row['study_id'].values[0])
                pred_boxes.append(new_boxes)
                pred_scores.append(scores)
                pred_labels.append(labels.astype(int))

#         return pd.DataFrame({'img_id':img_ids, 'study_id':study_ids, 'labels':pred_labels, 'scores':pred_scores, 'boxes':pred_boxes})
        return pd.DataFrame({'img_id':img_ids, 'labels':pred_labels, 'scores':pred_scores, 'boxes':pred_boxes})

## Model and Dataloader 

In [41]:
if effdet_pred:
    models = []
    model_name = 'tf_efficientnetv2_l_in21ft1k'
    for fold in range(cfg['fold_num']): 
        checkpoint_path = f'/kaggle/input/siim-covid19-efficientdet/{cfg["effdet_exp_id"]}/weights/best_loss_fold{fold}.pth'
#         models.append(load_obj_detection_model(model_name, f'/kaggle/input/siim-covid19-efficientdet/{cfg["effdet_exp_id"]}/weights/best_loss_fold{fold}.pth'))
        model = create_model(num_classes=1, image_size=cfg['img_size'], architecture=model_name, checkpoint_path=checkpoint_path)
        models.append(model)
    # test data
    det_batch_size = 8
    test_dataset = DetectDataset(test_df, test_dir, transform=get_test_transforms())

    test_loader = torch.utils.data.DataLoader(
        test_dataset, 
        batch_size=cfg['det_batch_size'],
        num_workers=cfg['num_workers'],
        shuffle=False,
        pin_memory=False,
        drop_last=False,
    )

In [42]:
if effdet_pred:
    study_preds_df = ensemble_predictions(test_loader)
    study_preds_df.head()

## Post Process

In [43]:
if effdet_pred:
    # create prediction string for a single image
    def create_image_prediction_string(row, opacity=True):
        pred_strings = []
        if opacity:
            if len(row['labels']) > 0:
                # if there are predicted labels
                for score,box in zip(row['scores'], row['boxes']):
                    x1,y1,x2,y2 = box
                    pred_strings.append("opacity {:.4f} {} {} {} {}".format(score, x1, y1, x2, y2))
                return ' '.join(pred_strings)
        # else - no predicted label (negative) or not opacity (none)
        return "none 1 0 0 1 1"

    def create_image_df(study_preds_df):
        image_ids = []
        pred_strings = []

        # for none predicted images
    #     none_df = test_df[test_df['image_level']==Configs.image_level[NONE]]
    #     for index, row in none_df.iterrows():
    #         image_ids.append(row['img_id']+'_image'),
    #         pred_strings.append(create_image_prediction_string(row, opacity=False))

        # for opacity predicted images
        for index, row in study_preds_df.iterrows():
            img_id = row['img_id']
            if '_image.png' in img_id:
                img_id = img_id.replace('_image.png', '')
            if not '_image' in img_id:
                image_ids.append(img_id + '_image')
            else:
                image_ids.append(img_id)
            pred_strings.append(create_image_prediction_string(row))

        return pd.DataFrame({'id':image_ids, 'PredictionString':pred_strings})


In [44]:
if effdet_pred:
    pred_df = create_image_df(study_preds_df)

## MMDetection Prediction

In [45]:
if mmdet_pred:
    # Check mmcv installation
    from mmcv.ops import get_compiling_cuda_version, get_compiler_version
    print(get_compiling_cuda_version())
    print(get_compiler_version())

    # Check MMDetection installation
    from mmdet.apis import set_random_seed

    # Imports
    import mmdet
    from mmdet.apis import set_random_seed
    from mmdet.datasets import build_dataset
    from mmdet.models import build_detector

    import mmcv
    from mmcv import Config
    from mmcv.runner import load_checkpoint
    from mmcv.parallel import MMDataParallel
    from mmdet.apis import inference_detector, init_detector, show_result_pyplot
    from mmdet.apis import single_gpu_test
    from mmdet.datasets import build_dataloader, build_dataset


In [46]:
if mmdet_pred:
    import cv2
    import matplotlib.pyplot as plt

    label2color = [[59, 238, 119]]
    IMAGE_DIMS = (512, 512)

    viz_labels =  ["Covid_Abnormality"]

    def plot_img(img, size=(18, 18), is_rgb=True, title="", cmap=None):
        plt.figure(figsize=size)
        plt.imshow(img, cmap=cmap)
        plt.suptitle(title)
        plt.show()

    def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title="", cmap=None, img_size=None):
        rows = len(imgs)//cols + 1
        fig = plt.figure(figsize=(cols*size, rows*size))
        for i, img in enumerate(imgs):
            if img_size is not None:
                img = cv2.resize(img, img_size)
            fig.add_subplot(rows, cols, i+1)
            plt.imshow(img, cmap=cmap)
        plt.suptitle(title)
        return fig

    def draw_bbox(image, box, label, color):   
        alpha = 0.1
        alpha_font = 0.6
        thickness = 8
        font_size = 2.0
        font_weight = 1
        overlay_bbox = image.copy()
        overlay_text = image.copy()
        output = image.copy()

        text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_weight)[0]
        cv2.rectangle(overlay_bbox, (box[0], box[1]), (box[2], box[3]),
                    color, -1)
        cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
        cv2.rectangle(overlay_text, (box[0], box[1]-18-text_height), (box[0]+text_width+8, box[1]),
                    (0, 0, 0), -1)
        cv2.addWeighted(overlay_text, alpha_font, output, 1 - alpha_font, 0, output)
        cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]),
                        color, thickness)
        cv2.putText(output, label.upper(), (box[0], box[1]-12),
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 255, 255), font_weight, cv2.LINE_AA)
        return output

    def draw_bbox_small(image, box, label, color):   
        alpha = 0.1
        alpha_text = 0.3
        thickness = 1
        font_size = 0.4
        overlay_bbox = image.copy()
        overlay_text = image.copy()
        output = image.copy()

        text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, font_size, thickness)[0]
        cv2.rectangle(overlay_bbox, (box[0], box[1]), (box[2], box[3]),
                    color, -1)
        cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
        cv2.rectangle(overlay_text, (box[0], box[1]-7-text_height), (box[0]+text_width+2, box[1]),
                    (0, 0, 0), -1)
        cv2.addWeighted(overlay_text, alpha_text, output, 1 - alpha_text, 0, output)
        cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]),
                        color, thickness)
        cv2.putText(output, label.upper(), (box[0], box[1]-5),
                cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 255, 255), thickness, cv2.LINE_AA)
        return output

In [47]:
if mmdet_pred:
    baseline_cfg_path = "/kaggle/input/siim-mmdetection-cascadercnn-weight-bias/job4_cascade_rcnn_x101_32x4d_fpn_1x_fold0/job4_cascade_rcnn_x101_32x4d_fpn_1x_coco.py"
    mmdet_cfg = Config.fromfile(baseline_cfg_path)

    mmdet_cfg.classes = ("Covid_Abnormality")
    mmdet_cfg.data.test.img_prefix = ''
    mmdet_cfg.data.test.classes = mmdet_cfg.classes

    # cfg.model.roi_head.bbox_head.num_classes = 1
    # cfg.model.bbox_head.num_classes = 1
    for head in mmdet_cfg.model.roi_head.bbox_head:
        head.num_classes = 1

    # Set seed thus the results are more reproducible
    mmdet_cfg.seed = 211
    set_random_seed(211, deterministic=False)
    mmdet_cfg.gpu_ids = [0]

    mmdet_cfg.data.test.pipeline=[
                dict(type='LoadImageFromFile'),
                dict(
                    type='MultiScaleFlipAug',
                    img_scale=(1333, 800),
                    flip=False,
                    transforms=[
                        dict(type='Resize', keep_ratio=True),
                        dict(type='RandomFlip', direction='horizontal'),
                        dict(
                            type='Normalize',
                            mean=[123.675, 116.28, 103.53],
                            std=[58.395, 57.12, 57.375],
                            to_rgb=True),
                        dict(type='Pad', size_divisor=32),
                        dict(type='DefaultFormatBundle'),
                        dict(type='Collect', keys=['img'])
                    ])
            ]

    mmdet_cfg.test_pipeline = [
                dict(type='LoadImageFromFile'),
                dict(
                    type='MultiScaleFlipAug',
                    img_scale=(1333, 800),
                    flip=False,
                    transforms=[
                        dict(type='Resize', keep_ratio=True),
                        dict(type='RandomFlip', direction='horizontal'),
                        dict(
                            type='Normalize',
                            mean=[123.675, 116.28, 103.53],
                            std=[58.395, 57.12, 57.375],
                            to_rgb=True),
                        dict(type='Pad', size_divisor=32),
                        dict(type='DefaultFormatBundle'),
                        dict(type='Collect', keys=['img'])
                    ])
            ]

    # cfg.data.samples_per_gpu = 4
    # cfg.data.workers_per_gpu = 4
    # cfg.model.test_cfg.nms.iou_threshold = 0.3
    mmdet_cfg.model.test_cfg.rcnn.score_thr = 0.001

    WEIGHTS_FILE = '/kaggle/input/siim-mmdetection-cascadercnn-weight-bias/job4_cascade_rcnn_x101_32x4d_fpn_1x_fold0/epoch_10.pth'
    options = dict(classes = ("Covid_Abnormality"))
    model = init_detector(mmdet_cfg, WEIGHTS_FILE, device='cuda:0')

In [48]:
if mmdet_pred:
    from ensemble_boxes import weighted_boxes_fusion, nms

    viz_images = []
    results = []
    score_threshold = mmdet_cfg.model.test_cfg.rcnn.score_thr

    def format_pred(boxes: np.ndarray, scores: np.ndarray, labels: np.ndarray) -> str:
        pred_strings = []
        label_str = ['opacity']
        for label, score, bbox in zip(labels, scores, boxes):
            xmin, ymin, xmax, ymax = bbox.astype(np.int64)
            pred_strings.append(f"{label_str[int(label)]} {score:.16f} {xmin} {ymin} {xmax} {ymax}")
        return " ".join(pred_strings)

    model.to(device)
    model.eval()

    viz_images = []

    with torch.no_grad():
        # test_df
        for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
            original_H, original_W = (int(row.dim0), int(row.dim1))
            predictions = inference_detector(model, row.image_path)
            boxes, scores, labels = (list(), list(), list())

            for k, cls_result in enumerate(predictions):
    #             print("cls_result", cls_result)
                if cls_result.size != 0:
                    if len(labels)==0:
                        boxes = np.array(cls_result[:, :4])
                        scores = np.array(cls_result[:, 4])
                        labels = np.array([k]*len(cls_result[:, 4]))
                    else:    
                        boxes = np.concatenate((boxes, np.array(cls_result[:, :4])))
                        scores = np.concatenate((scores, np.array(cls_result[:, 4])))
                        labels = np.concatenate((labels, [k]*len(cls_result[:, 4])))

                if fast_sub:
                    img_viz = cv2.imread(row.image_path)
                    for box, label, score in zip(boxes, labels, scores):
                        color = label2color[int(label)]
                        img_viz = draw_bbox_small(img_viz, box.astype(np.int32), f'opacity_{score:.4f}', color)
                    viz_images.append(img_viz)

            indexes = np.where(scores > score_threshold)
    #         print(indexes)
            boxes = boxes[indexes]
            scores = scores[indexes]
            labels = labels[indexes]

            if len(labels) != 0:
                h_ratio = original_H/IMAGE_DIMS[0]
                w_ratio = original_W/IMAGE_DIMS[1]
                boxes[:, [0, 2]] *= w_ratio
                boxes[:, [1, 3]] *= h_ratio

                result = {
                    "id": row.id,
                    "PredictionString": format_pred(
                        boxes, scores, labels
                    ),
                }

                results.append(result)
    del model
    gc.collect()

    pred_df = pd.DataFrame(results, columns=['id', 'PredictionString'])

    if fast_sub:
        display(pred_df.sample(2))
        # Plot sample images
        plot_imgs(viz_images, cmap=None)
        plt.savefig('viz_fig_siim.png', bbox_inches='tight')
        plt.show()

In [49]:
# if mmdet_pred:
#     detection_df = detection_df.merge(image_df[['image_id', 'none']].rename({'image_id':'id'}, axis=1),
#                                   on='id', how='left')

#     for i in range(detection_df.shape[0]):
#         if detection_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
#             detection_df.loc[i,'PredictionString'] = detection_df.loc[i,'PredictionString'] + ' none ' + str(detection_df.loc[i,'none']) + ' 0 0 1 1'
#     detection_df = detection_df[['id', 'PredictionString']]

#     results_df = study_df[['study_id', 'PredictionString']].rename({'study_id':'id'}, axis=1)
#     results_df = results_df.append(detection_df[['id', 'PredictionString']])


In [50]:
if yolov5_pred or effdet_pred or mmdet_pred:
    pred_df.head(5)

In [51]:
if yolov5_pred or effdet_pred or mmdet_pred:
    test_df = test_df.drop(['PredictionString'], axis=1)
    sub_df = pd.merge(test_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
    sub_df = sub_df[['id', 'PredictionString']]
    for i in range(sub_df.shape[0]):
        if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
            continue
        sub_df_split = sub_df.loc[i,'PredictionString'].split()
        sub_df_list = []
        for j in range(int(len(sub_df_split) / 6)):
            sub_df_list.append('opacity')
            sub_df_list.append(sub_df_split[6 * j + 1])
            sub_df_list.append(sub_df_split[6 * j + 2])
            sub_df_list.append(sub_df_split[6 * j + 3])
            sub_df_list.append(sub_df_split[6 * j + 4])
            sub_df_list.append(sub_df_split[6 * j + 5])
        sub_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)
    sub_df['none'] = df_2class['none'] 
    for i in range(sub_df.shape[0]):
        if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
            sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + str(sub_df.loc[i,'none']) + ' 0 0 1 1'
    sub_df = sub_df[['id', 'PredictionString']]  

In [52]:
if yolov5_pred or effdet_pred or mmdet_pred:
    print('Predicting with YOLOv5 or Effdet or MMDetection')
    df_study = df_study[:study_len]
    df_study = df_study.append(sub_df).reset_index(drop=True)
    del sub_df
    gc.collect()

Predicting with YOLOv5 or Effdet or MMDetection


In [53]:
df_study.to_csv('/kaggle/working/submission.csv',index = False)

In [54]:
if not is_local and yolov5_pred:
    shutil.rmtree('/kaggle/working/yolov5')
#         !rm -r ./test/study/
if not is_local and mmdet_pred:
    !rm -r /kaggle/working/mmdetection

In [55]:
df_study.head(5)

,id,PredictionString
0,00086460a852_study,negative 0.0013276135 0 0 1 1 typical 0.004610...
1,000c9c05fd14_study,negative 0.0037268507 0 0 1 1 typical 0.002002...
2,65761e66de9f_image,opacity 0.235229000449 701 769 1239 1415 opaci...
3,51759b5579bc_image,opacity 0.222900003195 200 1849 1008 2477 none...
